# Finite size DMRG algo for Heisenberg spin chain system 
## The Hamiltonian of the Heisenberg spin chain system is given below
<div class='math'>
    \begin{equation}
    \hat{{\cal H}}=\sum_{i=1}^{N-1} \vec{\sigma}_i.\vec{\sigma}_{i+1}
    \end{equation}
</div>

In [2]:
# Importing libraries
import numpy as np
from numpy import kron
from scipy import identity
from scipy.sparse.linalg import eigsh
import scipy.linalg.lapack as la
import math
from collections import namedtuple 

### One-qubit operator
Constructing the following one-qubit operator
<div class="math">
\begin{align}
  \sigma^z=\begin{bmatrix}
    1 & 0\\
    0 & -1
    \end{bmatrix}, && \sigma^+=\begin{bmatrix}
    0 & 2\\
    0 & 0
    \end{bmatrix}, && \sigma^-=\begin{bmatrix}
    0 & 0\\
    2 & 0
    \end{bmatrix}, && I_2=\begin{bmatrix}
    1 & 0\\
    0 & 1
    \end{bmatrix}.
\end{align}
</div>

In [3]:
# Step1: constructing one-qubit operator
splus=np.zeros([2,2], dtype='float64')
sminus=np.zeros([2,2],dtype='float64')
sz=np.zeros([2,2],dtype='float64')
splus[0,1]=2
sminus[1,0]=2
sz[0,0]=1
sz[1,1]=-1
ide=np.eye(2,dtype='float64')

### Constructing the two-qubit left Hamiltonian
<div class="math">
\begin{equation}
  \hat{{\cal H}}_{L2} = \sigma^z\otimes\sigma^z+\frac{1}{2}\left[\sigma^+\otimes \sigma^- + \sigma^-\otimes \sigma^+ \right]
\end{equation}
</div>

In [4]:
# Step2: Constructing two-qubit Hamiltonian
H2=kron(sz,sz)+0.5*(kron(splus,sminus)+kron(sminus,splus))

#### The namedtuple is a factory function for creating tuple subclasses with named fields. More details regarding factory function you can watch the following youtube [video](https://youtu.be/JSo2f-hhMbE). The general structure of namedtuple is given below

<span style="color:red;font-style: italic">
collections.namedtuple(typename, field_names, *, rename=False, defaults=None, module=None)
</span>

**Returns a new tuple subclass named *typename*. The new subclass is used to create tuple-like objects that have fields accessible by attribute lookup as well as being indexable and iterable. Instances of the subclass also have a helpful docstring (with *typename* and *field_names*) and a helpful __repr__() method which lists the tuple contents in a name=value format.**

In [5]:
Block = namedtuple("Block", ["number_spins", "basis_size", "operator_dict"])
increment_block=namedtuple("increment_block", ["number_spins", "basis_size", "operator_dict"])
intial_block=Block(number_spins=2,basis_size=4,\
                   operator_dict={'ham':H2,'sz':kron(ide,sz),'sp':kron(ide,splus),'sm':kron(ide,sminus),\
                                  'id':kron(ide,ide)})


### The block_increment method does the following work,
1. Work 1: <div class="math">
    \begin{eqnarray}
    \hat{{\cal H}}_{L~i+1} = \hat{{\cal H}}_{L~i} \otimes I_2+\widetilde{\sigma}^z_{i}\otimes\sigma^z + \frac{1}{2}\left[\widetilde{\sigma}_{i}^+\otimes \sigma^- + \widetilde{\sigma}_{i}^-\otimes \sigma^+\right]
    \end{eqnarray}
    </div>
    where $\widetilde{\sigma}^z_{i}=I_{2^{i-1}}\otimes \sigma^z$ and $\widetilde{\sigma}^\pm_{i}=I_{2^{i-1}}\otimes \sigma^\pm$
    
2. Work 2 : In the namedtuple output_block store the number of spins, basis size, and all the operators of the incremented left block ($\hat{{\cal H}}_{L~i+1}$).

In [6]:
def block_increment(input_block):
    # Work 1
    HL_new=kron(input_block.operator_dict['ham'],ide)\
    +kron(input_block.operator_dict['sz'],sz)+\
    0.5*(kron(input_block.operator_dict['sp'],sminus)+kron(input_block.operator_dict['sm'],splus))
    enlarged_operator_dictionary={'ham':HL_new,'sz':kron(input_block.operator_dict['id'],sz),\
                                 'sp':kron(input_block.operator_dict['id'],splus),\
                                 'sm':kron(input_block.operator_dict['id'],sminus),\
                                 'id':kron(input_block.operator_dict['id'],ide)}
    # Work 2
    output_block=increment_block(number_spins=input_block.number_spins+1,basis_size=input_block.basis_size*2,\
                                operator_dict=enlarged_operator_dictionary)
    return(output_block)

#### Before discussing the workflow of the dmrg_step method, we first briefly explain what do we mean by system block (sysblock) and environmental block (envblock). When performing the infinite size DMRG step, the sysblock = envblock = $\hat{{\cal H}}_{L~i+1}$=$\hat{{\cal H}}_{R~i+2}$. When performing left to right sweep in that case sysblock = $\hat{{\cal H}}_{L~i+1}$, and envblock = $\hat{{\cal H}}_{R~i+2}$, and during right to left sweep in that case sysblock = $\hat{{\cal H}}_{R~i+2}$ and envblock = $\hat{{\cal H}}_{L~i+1}$.

#### The dmrg_step method has the following work flow,
1. Work 1: <div class="math">
    \begin{eqnarray}
    \hat{{\cal H}}_{s^\prime}^\prime = \hat{{\cal H}}_{s} \otimes I_2+\widetilde{\sigma}^z_{s}\otimes\sigma^z + \frac{1}{2}\left[\widetilde{\sigma}_{s}^+\otimes \sigma^- + \widetilde{\sigma}_{s}^-\otimes \sigma^+\right]
    \end{eqnarray}
    </div>
    where $\widetilde{\sigma}^z_{s}=I_{2^{d_s-1}}\otimes \sigma^z$ and $\widetilde{\sigma}^\pm_{s}=I_{2^{d_s-1}}\otimes \sigma^\pm$, and $d_s$ is the dimension of the $\hat{{\cal H}}_{s}$
2. Work 2: If $\hat{{\cal H}}_{s}==\hat{{\cal H}}_{e}$ then $\hat{{\cal H}}_{e^\prime}^\prime=\hat{{\cal H}}_{s^\prime}^\prime$, otherwise, $\hat{{\cal H}}_{e^\prime}^\prime=\hat{{\cal H}}_{e}$. Case 1 is primarily is for infinite size dmrg step.
3. Work 3: The **new_block** stores the incremented spins and the basis size of $\hat{{\cal H}}_{s^\prime}^\prime$, and all the transformed operators or non-transformed operators ($\widetilde{\sigma}^z_{s^\prime}$, $\widetilde{\sigma}^\pm_{s^\prime}$, $I_{s^\prime}^\prime$ and $\hat{{\cal H}}_{s^\prime}^\prime$) of the $\hat{{\cal H}}_{s^\prime}^\prime$ Hilbert space.
4. Work 4: If $dim.(\hat{{\cal H}}_{s}^\prime)==m$ then do the following,

    4.1 Work 4.1: Construct the super Hamiltonian, <div class="math">
    \begin{eqnarray}
    \hat{{\cal H}}_{sup}=\hat{{\cal H}}_{s^\prime}^\prime\otimes I_{d_e^\prime}+I_{d_s^\prime}\otimes\hat{{\cal H}}_{e^\prime}^\prime + \widetilde{\sigma}^z_{s^\prime}\otimes \widetilde{\sigma}^z_{e^\prime}+\widetilde{\sigma}^+_{s^\prime}\otimes \widetilde{\sigma}^-_{e^\prime}+\widetilde{\sigma}^-_{s^\prime}\otimes \widetilde{\sigma}^+_{e^\prime}
    \end{eqnarray}
    </div>
   4.2 Work 4.2: Diagonalize the $\hat{{\cal H}}_{sup}$ matrix, and stores the ground state (eigvec) in $|\psi_g\rangle_{d\times 1}$ where $d={d^\prime_s\times d_e^\prime}$, and print the ground state energy eigenvalue (eigval) $E_g$.
   
   4.3 Work 4.3: Partial Trace out the environment block from $|\psi_g\rangle_{d \times 1}$, and it can be simply done by first constructing the matrix $\rho$ by reshaping $|\psi_g\rangle_{d \times 1}$ into a matrix of dimension ${d^\prime_s\times d_e^\prime}$, and finally the reduced density matrix is written as,<div class="math">
    \begin{equation}
    \rho^\prime_{d_s^\prime \times d_s^\prime} = \rho_{d_s^\prime \times d_e^\prime}\times \rho^\dagger_{d_e^\prime \times d_s^\prime}
    \end{equation}
    </div>
   4.4 Work 4.4: Diagonalize $\rho^\prime$, and keep only those eigenvectors who have the largest m eigenvalues. Using these eigenvectors we create the transformation matrix,<div class="math">
    \begin{equation}
    \hat{T}_f= \sum_{i=1}^m \lambda_i |\lambda_i\rangle \langle \lambda_i|
    \end{equation}
    </div>
   4.5 Work 4.5: Transformation of all the operators,<div class="math">
    \begin{equation}
    \hat{O}^\prime_{new}=\hat{T}_f^\dagger \hat{O}^\prime \hat{T}_f
    \end{equation}
    </div>
    where $\hat{O}^\prime = \{ \widetilde{\sigma}^z_{s^\prime}, \widetilde{\sigma}^\pm_{s^\prime}, I_{s^\prime}^\prime,\hat{{\cal H}}_{s^\prime}^\prime \}$
    
   4.6 Work 4.6: In the **new_block** stores the number of spins and the basis size of $\hat{{\cal H}}_{s^\prime}^\prime$, and the transformed operators $\hat{O}^\prime_{new}$
   
5. Work 5: Finally it returns the **new_block**.

In [7]:
def dmrg_step(sysblock,envblock,m):
    # Work 1
    sysblock_new=block_increment(sysblock)
    # Work 2
    if sysblock is envblock:  # no need to recalculate a second time
        envblock_new = sysblock_new
    else:
        envblock_new = envblock
    # Work 3
    new_block=sysblock_new
    # Work 4
    if sysblock_new.basis_size > m:
        # Work 4.1
        super_ham=kron(sysblock_new.operator_dict['ham'],envblock_new.operator_dict['id'])+\
        kron(sysblock_new.operator_dict['id'],envblock_new.operator_dict['ham'])+\
        kron(sysblock_new.operator_dict['sz'],envblock_new.operator_dict['sz'])+\
        0.5*(kron(sysblock_new.operator_dict['sp'],envblock_new.operator_dict['sm'])+\
             kron(sysblock_new.operator_dict['sm'],envblock_new.operator_dict['sp']))
        # Work 4.2 
        eigval, eigvec = eigsh(super_ham, k=1,which='SA')
        print(eigval/4)
        # Work 4.3
        eigvec=eigvec.reshape(sysblock_new.basis_size,envblock_new.basis_size)
        rho_red=np.matmul(eigvec,np.transpose(eigvec))
        # Work 4.4
        eval_red,evec_red,info=la.dsyev(rho_red)
        tf_matrix=evec_red[:,np.shape(rho_red)[0]-m:np.shape(rho_red)[0]]
        # Work 4.5
        new_dictionary={'ham':np.matmul(np.matmul(np.transpose(tf_matrix),sysblock_new.operator_dict['ham']),\
                                        tf_matrix),\
                       'sz':np.matmul(np.matmul(np.transpose(tf_matrix),sysblock_new.operator_dict['sz']),\
                                      tf_matrix),\
                       'sp':np.matmul(np.matmul(np.transpose(tf_matrix),sysblock_new.operator_dict['sp']),\
                                      tf_matrix),\
                       'sm':np.matmul(np.matmul(np.transpose(tf_matrix),sysblock_new.operator_dict['sm']),\
                                      tf_matrix),\
                       'id':np.matmul(np.matmul(np.transpose(tf_matrix),sysblock_new.operator_dict['id']),\
                                      tf_matrix)}
        # Work 4.6
        new_block=Block(number_spins=sysblock_new.number_spins,basis_size=m,operator_dict=new_dictionary)
    # Work 5
    return(new_block)

In [8]:
def finite_dmrg(N,m,sweeps):
    """
    Performs the finite size dmrg function
    Attributes
        N: number of spins
        m: number of largest eigenvalues to stored
        sweep: number of sweeps
    Returns no parameter, only prints the ground state energy eigenvalues.
    """
    # LR: Left Right
    LR_operators={}
    # Intialize with the two-qubit Hamiltonian
    BlockL=intial_block
    # sys: system block, and env: environment block
    LR_operators['sys',BlockL.number_spins]=BlockL
    LR_operators['env',BlockL.number_spins]=BlockL
    # infinite size dmrg step
    while BlockL.number_spins<N/2:
        BlockL=dmrg_step(BlockL,BlockL,m)
        LR_operators['sys',BlockL.number_spins]=BlockL
        LR_operators['env',BlockL.number_spins]=BlockL
    # Sweep part
    system_label='sys'
    env_label='env'
    system_block=BlockL
    del BlockL
    for i in range(1,sweeps+1):
        env_block=LR_operators[env_label,N-system_block.number_spins-1]
        system_block=dmrg_step(system_block,env_block,m)
        if env_block.number_spins == 2:
            # We've come to the end of the chain, so we reverse course.
            system_block, env_block = env_block, system_block
            system_label, env_label = env_label, system_label
        LR_operators[system_label, system_block.number_spins] = system_block

In [9]:
finite_dmrg(10,10,10)

[-3.3749326]
[-4.25802886]
[-4.25802837]
[-4.25802843]
[-4.25802843]
[-4.25802843]
[-4.25802843]
[-4.25802843]
[-4.25802848]
[-4.25802848]
